In [ ]:
import numpy as np
from pyqubo import Binary
import pyqubo
import neal
import matplotlib.pyplot as plt 
from sympy import *
from datetime import datetime
import pandas as pd


## Input Data 

In [ ]:
#input data 


#Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].to_numpy()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].to_numpy()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].to_numpy()

n_assets = len(asset_quantity)

#Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].to_numpy()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].to_numpy()

n_accounts = len(account_exposure)

#haircuts 
haircuts = pd.read_csv("data/sample_haircuts.csv", header=None).to_numpy()/100

#Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

# Penalty Terms

In [ ]:

#Penalty Terms
lambda_cost_fn = 1e5
lambda_consistency = 1
lambda_exposure = 1



# Defining the QUBO

In [ ]:
# Binary variables

n_binary = 7
max_binary_value = 2**(n_binary) - 1 

binary_coeff_vector = np.zeros(n_binary)

for i in range(n_binary):
    binary_coeff_vector[i] = 2**i


#Decision variable

binary_variables = []

for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            globals()["x" + str(i + 1) + str(j + 1) + str(k+1)] = Binary("x" + str(i + 1) + str(j + 1) + str(k+1))
            binary_variables.append(globals()["x" + str(i + 1) + str(j + 1) + str(k + 1)])


#Slack variable for consistency 
num_slack_consistency_arr = np.zeros(shape=(n_assets))

for i in range(n_assets):
    num_slack_consistency_arr[i] = int(np.ceil(np.log2(max_binary_value)))


consistency_slack_variables_arr = [] 

for count, i in enumerate(num_slack_consistency_arr): 
    consistency_slack_variables_arr.append(pyqubo.Array.create("consistency_asset" + str(count + 1) + "_slack", shape = int(i), vartype = "BINARY"))

    

# Constructing the QUBO equation 

This involves the QUBO objective function, the consistency contraints, the exposure constraints.

In [ ]:

#QUBO objective function 

cost_fn = 0

for i in range(n_assets):
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        cost_fn += aux_term1*cost_factor_matrix[i][j]

cost_fn /= max_binary_value


In [ ]:
# Consistency constraint 

consistency_penalty_term = 0

#slack part
consistency_penalty_slack = []

for i in consistency_slack_variables_arr:
    aux_term = 0
    for exponent,j in enumerate(i):
        aux_term += (2**exponent)*j
    consistency_penalty_slack.append(aux_term)


#decision variable part
for i in range(n_assets):
    aux_term2 = 0
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1)
    consistency_penalty_term += (aux_term2 - max_binary_value + consistency_penalty_slack[i])**2

In [ ]:
#Exposure constraint

exposure_penalty_term = 0

for j in range(n_accounts):
    aux_term2 = 0
    for i in range(n_assets):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1*asset_quantity[i]*asset_value[i]*haircuts[i][j])/max_binary_value
    exposure_penalty_term += (aux_term2 - account_exposure[j]) **2 

exposure_penalty_term /= np.mean(asset_quantity*asset_value)

In [ ]:


#Final QUBO
QUBO_equation =lambda_cost_fn*cost_fn + lambda_consistency*consistency_penalty_term  + lambda_exposure*exposure_penalty_term 

#compile model
model = QUBO_equation.compile()
qubo, offset = model.to_qubo()


# Run Solver

In [ ]:
#Run Solver

bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()
num_reads = 100
num_sweeps = 1000
sampleset = sa.sample(bqm, num_reads=num_reads, num_sweeps=num_sweeps)

#Get best sample
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
results = best_sample.sample
pprint(f"The minimum energy is: {best_sample.energy}")


# Verify Results

In [ ]:

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        decimal = 0
        for k in range(n_binary):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]
    allocation_solution[i] = asset


In [ ]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")  

In [ ]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]:,} required") 
    print(f"{(exposure_solution[i]/account_exposure[i] - 1) *100.:0.2f}%")

# Export to csv

In [ ]:
#Saving to an array
Q_value_balanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        decimal = 0
        for k in range(n_binary):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        Q_value_balanced_py_qubo[i][j] = decimal/max_binary_value

pprint(f"Objective value: {sum(sum(Q_value_balanced_py_qubo*cost_factor_matrix))}")

In [ ]:
#Uncomment line below to save file to csv

#pd.DataFrame(Q_value_balanced_py_qubo).to_csv("Q_value_balanced_py_qubo_no_single_lim1.csv", header=None, index=None)
